# 00 - Extraccion de datos mediante API de Mercado Libre

## Importación de librerias

In [43]:
import pandas as pd
import numpy as np
import json
import requests

from datetime import datetime, timedelta, timezone, date
pd.options.display.max_columns=None

try:
    from pyprojroot import here
    print("Libreria: 'pyprojroot' esta instalada y se cargo correctamente")
except ModuleNotFoundError:
    print("Libreria: 'pyprojroot' no esta instalada, se debe instalar")
    !pip install pyprojroot

import matplotlib.pyplot as plt
import os

try:
    import seaborn as sns
    print("Libreria: 'seaborn' esta instalada y se cargo correctamente")
except ModuleNotFoundError:
    print("Libreria: 'seaborn' no esta instalada, se debe instalar")
    !pip install seaborn


Libreria: 'pyprojroot' esta instalada y se cargo correctamente
Libreria: 'seaborn' esta instalada y se cargo correctamente


In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
path_secrets = "/content/drive/MyDrive/Austral/Maestria/Tesis/Codigo/secrets/"

## Definicion de funciones a utilizar para obtener los datos

In [46]:
def auth_token():
    with open(path_secrets + 'secrets.json') as f:
        secrets = json.load(f)
    
    print(f'http://auth.mercadolibre.com.ar/authorization?response_type=code&client_id={secrets["client_id"]}&redirect_uri={secrets["redirect_uri"]}')
    codigo = input('Codigo :')
    url = 'https://api.mercadolibre.com/oauth/token'
    response = requests.post(url, data = {
        
        'grant_type' : 'authorization_code',
        'client_id': secrets['client_id'],
        'client_secret': secrets['client_secret'],
         'code' : codigo,
        'redirect_uri': secrets['redirect_uri']
    })
    print(response)
    return response.json()['access_token']

meli_auth_token = None

def get_response(url):
    global meli_auth_token
    
    try:
        if meli_auth_token is None and 'mercadolibre' in url:
            meli_auth_token = auth_token()
        response = requests.get(url, timeout = 4,
                                   headers={'Authorization': f'Bearer {meli_auth_token}'})
        response.raise_for_status()
        return response
    except requests.exceptions.HTTPError as errh:
        print ("Http Error:",errh)
    except requests.exceptions.ConnectionError as errc:
        print ("Error Connecting:",errc)
    except requests.exceptions.Timeout as errt:
        print ("Timeout Error:",errt)
    except requests.exceptions.RequestException as err:
        print ("OOps: Something Else",err)
        
#Definimos una función para traer el elemento que queremos
def traerdato(elemento,rama,subrama,valor='value_name'):
    indices=[]
    for i,s in enumerate(elemento[rama]):
        for j in s:
            if subrama in str(s[j]):
                indices.append([i,s])
    if len(indices) == 0:
        return 'Sin Datos'
    else:
        return indices[0][1][valor]


In [47]:
meli_auth_token = auth_token()

http://auth.mercadolibre.com.ar/authorization?response_type=code&client_id=8800815426780367&redirect_uri=https://www.google.com.ar/
Codigo :TG-62f5730f0e510500016f952b-216145475
<Response [200]>


## Armado del Dataset

Para armar el dataset descargaremos para Mercado Libre Argentina laptops publicadas. La categoría se llama: MLA1652

In [48]:
url='https://api.mercadolibre.com/sites/MLA/search?category=MLA1652&search_type=scan#json'

r = get_response(url) 
maximum = int(str(r.json()["paging"]["total"])) #Guardamos el número máximo de resultados para luego stoppear nuestro loop

print(f'Encontramos {maximum} resultados para nuestra consulta')

Encontramos 33918 resultados para nuestra consulta


In [49]:
offset_max = maximum / 50
offset_max

678.36

In [50]:
offset = 0
item_list = []

while r.status_code == 200 and offset <= round(offset_max):
    url=f'https://api.mercadolibre.com/sites/MLA/search?category=MLA1652&offset={offset}#json'
    r = get_response(url)
    if r is not None:
      data = r.json()
      length = len(data['results'])
      for i in range(length):
          item_id = data['results'][i]['id']
          item_list.append(item_id)
      print("Porcentaje de completitud: {:0.2%}".format(offset/maximum),end='\r')

    offset += 1
          
len(item_list)

33835

In [52]:
final_list = []
for i in range(len(item_list)):
    item="https://api.mercadolibre.com/items/{}".format(item_list[i])
    item_add = requests.get(item)
    item_add = item_add.json()
    final_list.append(item_add)
    print("Porcentaje de completitud: {:0.2%}".format((i+1)/len(item_list)),end='\r')

In [ ]:
print(len(item_list))
print(len(set(item_list)))

In [56]:
data = pd.json_normalize(final_list)
data.head(3)

,id,site_id,title,subtitle,seller_id,category_id,official_store_id,price,base_price,original_price,currency_id,initial_quantity,available_quantity,sold_quantity,sale_terms,buying_mode,listing_type_id,start_time,stop_time,condition,permalink,thumbnail_id,thumbnail,secure_thumbnail,pictures,video_id,descriptions,accepts_mercadopago,non_mercado_pago_payment_methods,international_delivery_mode,seller_contact,coverage_areas,attributes,warnings,listing_source,variations,status,sub_status,tags,warranty,catalog_product_id,domain_id,parent_item_id,differential_pricing,deal_ids,automatic_relist,date_created,last_updated,health,catalog_listing,channels,shipping.mode,shipping.methods,shipping.tags,shipping.dimensions,shipping.local_pick_up,shipping.free_shipping,shipping.logistic_type,shipping.store_pick_up,seller_address.city.id,seller_address.city.name,seller_address.state.id,seller_address.state.name,seller_address.country.id,seller_address.country.name,seller_address.search_location.neighborhood.id,seller_address.search_location.neighborhood.name,seller_address.search_location.city.id,seller_address.search_location.city.name,seller_address.search_location.state.id,seller_address.search_location.state.name,seller_address.id,shipping.free_methods
0,MLA1129251702,MLA,Notebook Hp Amd Ryzen 7 5700u 12gb 256gb Ssd 1...,None,174556876,MLA1652,NaN,164899.0,164899.0,179999.0,ARS,43,1,5,"[{'id': 'WARRANTY_TYPE', 'name': 'Tipo de gara...",buy_it_now,gold_special,2022-03-25T16:19:38.000Z,2042-03-20T04:00:00.000Z,new,https://articulo.mercadolibre.com.ar/MLA-11292...,975457-MLA49471206711_032022,http://http2.mlstatic.com/D_975457-MLA49471206...,https://http2.mlstatic.com/D_975457-MLA4947120...,"[{'id': '975457-MLA49471206711_032022', 'url':...",None,[],True,[],none,None,[],"[{'id': 'BATTERY_LIFE', 'name': 'Duración de l...",[],,"[{'id': 174343023835, 'price': 164899.0, 'attr...",active,[],"[deal_of_the_day, extended_warranty_eligible, ...",Garantía de fábrica: 12 meses,None,MLA-NOTEBOOKS,None,None,"[MLA8258, MLA12440, MLA6015, MLA12436]",False,2022-03-25T16:19:38.000Z,2022-08-11T20:54:11.550Z,0.9,False,"[marketplace, mshops]",me2,[],"[self_service_in, mandatory_free_shipping]",None,True,True,cross_docking,False,TUxBQlZJTDQyMjBa,Villa Crespo,AR-C,Capital Federal,AR,Argentina,TUxBQlZJTDQyMjBa,Villa Crespo,TUxBQ0NBUGZlZG1sYQ,Capital Federal,TUxBUENBUGw3M2E1,Capital Federal,155782338,NaN
1,MLA1147028688,MLA,"Notebook Hp 15-ef2126wm Spruce Blue 15.6 , Amd...",None,158169010,MLA1652,NaN,129499.0,129499.0,NaN,ARS,42,0,25,"[{'id': 'INVOICE', 'name': 'Facturación', 'val...",buy_it_now,gold_special,2022-07-15T15:09:59.000Z,2042-07-10T04:00:00.000Z,new,https://articulo.mercadolibre.com.ar/MLA-11470...,945528-MLA49251126700_032022,http://http2.mlstatic.com/D_945528-MLA49251126...,https://http2.mlstatic.com/D_945528-MLA4925112...,"[{'id': '945528-MLA49251126700_032022', 'url':...",None,[],True,[],none,None,[],"[{'id': 'ASPECT_RATIO', 'name': 'Relación de a...",[],,[],paused,[out_of_stock],"[extended_warranty_eligible, good_quality_pict...",Garantía del vendedor: 6 meses,MLA18717712,MLA-NOTEBOOKS,None,None,[],False,2022-07-15T15:10:00.000Z,2022-08-11T21:29:46.849Z,NaN,True,[marketplace],me2,[],"[self_service_in, fbm_in_process, mandatory_fr...",None,True,True,xd_drop_off,False,TUxBQ0NBUGNiZGQx,Córdoba,AR-X,Córdoba,AR,Argentina,NaN,NaN,TUxBQ0NBUGNiZGQx,Córdoba,TUxBUENPUmFkZGIw,Córdoba,269536971,NaN
2,MLA1110784573,MLA,Notebook Noblex 14.1 Pantalla Hd Intel Celero...,None,185702920,MLA1652,NaN,66999.0,66999.0,NaN,ARS,81,1,50,"[{'id': 'WARRANTY_TIME', 'name': 'Tiempo de ga...",buy_it_now,gold_special,2021-11-02T17:35:08.000Z,2041-10-28T04:00:00.000Z,new,https://articulo.mercadolibre.com.ar/MLA-11107...,997940-MLA47767920611_102021,http://http2.mlstatic.com/D_997940-MLA47767920...,https://http2.mlstatic.com/D_997940-MLA4776792...,"[{'id': '997940-MLA47767920611_102021', 'url':...",None,[],True,[],none,None,[],"[{'id': 'ALPHANUMERIC_MODEL', 'name': 'Modelo ...",[],,[

In [57]:
data.title.value_counts()

Notebook Hp Amd Ryzen 5 5500 8gb 256gb Ssd 15.6 Windows 10                                                                              150
Notebook Lenovo Ip 3 15alc6 Ryzen 3 4gb 256gb W10s                                                                                      150
Notebook Noblex N14wce128 14.1'' Hd Intel Celeron 4gb 128gb                                                                             100
Notebook Hp I3-1125g4 4gb 128gb 14'' Fhd Win 10 Silver                                                                                  100
Notebook Bangho Core I7 11a Gen 16gb Ssd 480gb 15,6 Gamer Hf                                                                            100
                                                                                                                                       ... 
Notebook Noblex 14.1  Pantalla Hd Intel Celeron 4gb/128gb Ssd                                                                             3
Notebook Hp 15-ef212

In [60]:
path_save = "/content/drive/MyDrive/Austral/Maestria/Tesis/Data/"

In [61]:
data.to_csv(path_save + "datos_laptops.csv", index = False, sep = ";")